# Simple example

Given a function $f$, a strictly positive integer and a real number $x$ we want to define the function $I(f)$ such that: $I(f)(x)=\frac{1}{N} \sum_{n=0}^{N-1} f(\frac{n}{N-1} x) $

## code

In [1]:
def fjit(fun):
    'just-in-time compile a function by wrapping it in a singleton class'
    
    from numba import jitclass
    import time
    
    # the function is jitted first
    jitted_fun = njit(fun)

    # Generate a random class name like 'Singleton_Sat_Jan__2_18_08_32_2016'
    classname = 'Singleton_' + time.asctime().replace(' ','_').replace(':','_')
    
    # programmatically create a class equivalent to :
    # class Singleton_Sat_Jan__2_18_08_32_2016:
    #     def __init__(self): pass
    #     def value(self, x): return fj(x)
    
    def __init__(self): pass
    def value(self, x): return jitted_fun(x)
    SingletonClass = type(classname, (object,), {'__init__': __init__, 'value': value})
    
    # jit compile the class
    # spec is [] since we don't store attributes
    spec = []
    sc = jitclass(spec)(SingletonClass)
    
    # return a unique instance of the class
    return sc()

## usage

In [2]:
from numba import njit

In [3]:
@fjit
def f(x):
    return x**2

In [4]:
# f is now a "jitted" function (wrapped in a class)
f.value(3.)

9.0

In [5]:
from numpy import linspace
@njit
def function_of_a_function(f,x,N):
    xvec = linspace(0,x,N)
    t = 0.0
    for i in range(N):
        t += f.value(xvec[i])
    return t/N

In [6]:
%time function_of_a_function(f, 1, 10000)
%time function_of_a_function(f, 1, 10000)

CPU times: user 132 ms, sys: 0 ns, total: 132 ms
Wall time: 132 ms
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 240 µs


0.3333500016668329

# Performance cost

Here is a comparison of several ways to compute: $\frac{1}{N}\sum_{n=1}^N f(x)$ (the example is deliberately very simple so that automatic inlining can work).

The performance ranking is as follows (timing):

- copy and paste + jit    (x1)
- njitted method          (x1)
- jit-class method        (x3000)
- pure python             (x24000)

In [7]:
# functions

# pure python
def fun(x):
    return x**2

# jitclass
from numba import jitclass
spec = []
@jitclass(spec)
class SingletonClass:
    def __init__(self):
        pass
    def method(self, x):
        return x**2
sc = SingletonClass()

# jitted
@njit
def jitted_fun(x):
    return x**2

In [8]:
# functions of functions

def funfun_direct(x,N):
    t = 0.0
    for i in range(N):
        t = (x)**2
    return t

funfun_direct_jitted = njit(funfun_direct)

@njit
def funfun_jitted(x,N):
    t = 0.0
    for i in range(N):
        t = jitted_fun(x)
    return t

@njit
def funfun_class(f,x,N):
    t = 0.0
    for i in range(N):
        t = f.method(x)
    return t

In [9]:
N = 1000000
x = 1.0

In [10]:
# pure python code
%time funfun_direct(x, N)

CPU times: user 104 ms, sys: 0 ns, total: 104 ms
Wall time: 102 ms


1.0

In [11]:
# jit class
%time funfun_class(sc,x,N)
%time funfun_class(sc,x,N)

CPU times: user 124 ms, sys: 0 ns, total: 124 ms
Wall time: 123 ms
CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 14.7 ms


1.0

In [12]:
# jitted, manual inlining
%time funfun_direct_jitted(x, N)
%time funfun_direct_jitted(x, N)

CPU times: user 52 ms, sys: 0 ns, total: 52 ms
Wall time: 51.6 ms
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.72 µs


1.0

In [13]:
# jitted automatic inlining
%time funfun_jitted(x,N)
%time funfun_jitted(x,N)

CPU times: user 100 ms, sys: 0 ns, total: 100 ms
Wall time: 96.3 ms
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.72 µs


1.0

## strange inlining problem:

If x is not passed as an argument but defined in the function instead, inlining doesn't seem to happen.

In [14]:
@njit
def funfun_manual_inline(N):
    t = 0.0
    x = 2.0
    for i in range(N):
        t = (x)**2
    return t

@njit
def funfun_auto_inline(N):
    t = 0.0
    x = 2.0
    for i in range(N):
        t = jitted_fun(x)
    return t

In [15]:
%time funfun_manual_inline(N)
%time funfun_manual_inline(N)

CPU times: user 44 ms, sys: 0 ns, total: 44 ms
Wall time: 40.8 ms
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.72 µs


4.0

In [16]:
%time funfun_auto_inline(N)
%time funfun_auto_inline(N)

CPU times: user 48 ms, sys: 0 ns, total: 48 ms
Wall time: 48.1 ms
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 485 µs


4.0